In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [2]:
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

In [3]:
path_to_zip

'C:\\Users\\Masoud\\.keras\\datasets\\spa-eng.zip'

In [5]:
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt')
path_to_file

'C:\\Users\\Masoud\\.keras\\datasets\\spa-eng\\spa.txt'

In [6]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [7]:
def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<strat> ' + w + ' <end>'
    return w

In [8]:
en_sentence = "I have some friends to help."
preprocess_senetence(en_sentence)

'<strat> i have some friends to help . <end>'

In [9]:
sp_sentence = "Aún no he dicho nada."
preprocess_senetence(sp_sentence)

'<strat> aun no he dicho nada . <end>'

In [20]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)

In [22]:
print(create_dataset(path_to_file, 2))

[('<strat> go . <end>', '<strat> go . <end>'), ('<strat> ve . <end>', '<strat> vete . <end>')]


In [23]:
en, sp = create_dataset(path_to_file, None)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
len(en)

118964

In [25]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [26]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [27]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [ ]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [31]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]])

In [34]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [40]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [41]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <strat>
97  ....  ve
3  ....  .
2  ....  <end>


In [42]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [43]:
target_lang_tokenizer.word_index

{'<strat>': 1,
 '<end>': 2,
 '.': 3,
 'i': 4,
 'you': 5,
 '?': 6,
 'tom': 7,
 'is': 8,
 'it': 9,
 's': 10,
 'a': 11,
 'he': 12,
 't': 13,
 'the': 14,
 'we': 15,
 'm': 16,
 'me': 17,
 're': 18,
 'that': 19,
 'this': 20,
 'to': 21,
 'do': 22,
 'are': 23,
 'can': 24,
 'my': 25,
 'they': 26,
 'was': 27,
 'she': 28,
 'don': 29,
 'have': 30,
 'your': 31,
 'go': 32,
 'what': 33,
 'in': 34,
 'not': 35,
 'll': 36,
 'like': 37,
 'here': 38,
 'on': 39,
 'him': 40,
 'let': 41,
 'be': 42,
 'did': 43,
 'know': 44,
 'come': 45,
 'up': 46,
 'am': 47,
 'want': 48,
 'how': 49,
 ',': 50,
 'mary': 51,
 'get': 52,
 'who': 53,
 'very': 54,
 'now': 55,
 'need': 56,
 'has': 57,
 'please': 58,
 'no': 59,
 'there': 60,
 'help': 61,
 'her': 62,
 'love': 63,
 'at': 64,
 'see': 65,
 'just': 66,
 'out': 67,
 'his': 68,
 've': 69,
 'got': 70,
 'for': 71,
 'where': 72,
 'look': 73,
 'stop': 74,
 'one': 75,
 'us': 76,
 'good': 77,
 'car': 78,
 'too': 79,
 'so': 80,
 'all': 81,
 'why': 82,
 'will': 83,
 'an': 84,
 'hom

In [45]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [46]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [48]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [49]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [50]:
encoder

In [51]:
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [57]:
example_input_batch, example_target_batch = next(iter(dataset))

In [58]:
encoder(example_input_batch, simple_hidden)

(<tf.Tensor: shape=(64, 16, 1024), dtype=float32, numpy=
 array([[[-3.2697229e-03,  4.0113763e-03,  5.1406273e-03, ...,
           1.0146790e-03, -3.8364781e-03, -1.1349466e-03],
         [-3.1792778e-03, -7.1116970e-03, -6.3369814e-03, ...,
          -3.2275934e-03, -7.7813054e-03,  3.8633882e-03],
         [-8.1410613e-03, -4.3105278e-03, -1.0646933e-03, ...,
           3.9694924e-03,  9.2933923e-03, -7.1403543e-03],
         ...,
         [-6.1058099e-03, -4.8227562e-03, -4.0467968e-03, ...,
           1.2374989e-03, -1.4397904e-02, -6.4334450e-03],
         [-6.1226524e-03, -4.8243790e-03, -4.1263485e-03, ...,
           1.2099721e-03, -1.4365115e-02, -6.4720819e-03],
         [-6.1376989e-03, -4.8224572e-03, -4.1855406e-03, ...,
           1.1878673e-03, -1.4345031e-02, -6.4875246e-03]],
 
        [[-3.2697229e-03,  4.0113763e-03,  5.1406273e-03, ...,
           1.0146790e-03, -3.8364781e-03, -1.1349466e-03],
         [ 1.5482322e-03,  1.1249080e-02,  1.2172464e-02, ...,
         

In [66]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [67]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [68]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[-0.00457177, -0.00208527, -0.00184544, ...,  0.00028956,
         -0.00896624, -0.00370747],
        [ 0.00119451, -0.00183545,  0.00180924, ...,  0.00253158,
         -0.00733828, -0.00033673],
        [-0.00348966, -0.00281756, -0.00095681, ...,  0.00092489,
         -0.00979362, -0.00581631],
        ...,
        [-0.00199292,  0.00081374, -0.00246315, ..., -0.00045523,
         -0.00714276, -0.0041494 ],
        [-0.00395119, -0.00282114, -0.00196286, ..., -0.00089577,
         -0.01030923, -0.00516612],
        [-0.00334566, -0.00120477, -0.00217904, ...,  0.0016314 ,
         -0.00896375, -0.00446946]], dtype=float32)>,
 <tf.Tensor: shape=(64, 16, 1), dtype=float32, numpy=
 array([[[0.06392923],
         [0.06268611],
         [0.06234154],
         ...,
         [0.06266139],
         [0.06267209],
         [0.06267904]],
 
        [[0.06378199],
         [0.06346188],
         [0.06305225],
         ...,
         [0.

In [69]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [74]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [75]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [76]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[ 0.00207782, -0.00502978, -0.00237944, ...,  0.00313017,
         -0.00172754,  0.00146136],
        [ 0.00177008, -0.00503939, -0.0035195 , ...,  0.0023735 ,
         -0.00153377,  0.00171241],
        [ 0.00241185, -0.00491762, -0.0017625 , ...,  0.00286581,
         -0.00161236,  0.00060565],
        ...,
        [ 0.00229214, -0.00434617, -0.00229058, ...,  0.00248077,
         -0.00171415,  0.00084419],
        [ 0.00223128, -0.00512415, -0.00268228, ...,  0.00336393,
         -0.00156254,  0.00059456],
        [ 0.00227864, -0.0046121 , -0.00166312, ...,  0.00316759,
         -0.00198742,  0.00130165]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 0.00241017, -0.00440581,  0.00603626, ..., -0.00829175,
          0.00819389, -0.00392116],
        [ 0.0021082 , -0.0031887 ,  0.00607927, ..., -0.00898518,
          0.00700357, -0.00341018],
        [ 0.00337343, -0.00284369,  0.00614324,

In [77]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [82]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [83]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [84]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [86]:
EPOCH = 10
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

Epoch:  0
Loss:  1.8050487
Epoch:  0
Loss:  1.7023607
Epoch:  0
Loss:  1.7279295
Epoch:  0
Loss:  1.6006377
Epoch:  0
Loss:  1.7977968
Epoch:  0
Loss:  1.753999
Epoch:  0
Loss:  1.6647793
Epoch:  0
Loss:  1.7484028
Epoch:  0
Loss:  1.7394011
Epoch:  0
Loss:  1.6997505
Epoch:  0
Loss:  1.641506
Epoch:  0
Loss:  1.7192093
Epoch:  0
Loss:  1.7270237
Epoch:  0
Loss:  1.684694
Epoch:  0
Loss:  1.6944063
Epoch:  0
Loss:  1.6409185
Epoch:  0
Loss:  1.63401
Epoch:  0
Loss:  1.6236565
Epoch:  0
Loss:  1.6738955
Epoch:  0
Loss:  1.6176819
Epoch:  0
Loss:  1.6934607
Epoch:  0
Loss:  1.6982505
Epoch:  0
Loss:  1.7792361
Epoch:  0
Loss:  1.6934292
Epoch:  0
Loss:  1.6010917
Epoch:  0
Loss:  1.7052587
Epoch:  0
Loss:  1.7650273
Epoch:  0
Loss:  1.6578449
Epoch:  0
Loss:  1.7749679
Epoch:  0
Loss:  1.6513315
Epoch:  0
Loss:  1.6395652
Epoch:  0
Loss:  1.682965
Epoch:  0
Loss:  1.6450374
Epoch:  0
Loss:  1.6749302
Epoch:  0
Loss:  1.591013
Epoch:  0
Loss:  1.6969436
Epoch:  0
Loss:  1.7049822
Epoch:  

Epoch:  1
Loss:  1.1998388
Epoch:  1
Loss:  1.256511
Epoch:  1
Loss:  1.3727849
Epoch:  1
Loss:  1.2957364
Epoch:  1
Loss:  1.3056645
Epoch:  1
Loss:  1.3121142
Epoch:  1
Loss:  1.2610854
Epoch:  1
Loss:  1.2126713
Epoch:  1
Loss:  1.3527172
Epoch:  1
Loss:  1.1905887
Epoch:  1
Loss:  1.2295012
Epoch:  1
Loss:  1.3059638
Epoch:  1
Loss:  1.3041648
Epoch:  1
Loss:  1.3110658
Epoch:  1
Loss:  1.2901168
Epoch:  1
Loss:  1.2830583
Epoch:  1
Loss:  1.2291209
Epoch:  1
Loss:  1.293505
Epoch:  1
Loss:  1.2393779
Epoch:  1
Loss:  1.2541835
Epoch:  1
Loss:  1.2506571
Epoch:  1
Loss:  1.3376637
Epoch:  1
Loss:  1.2465574
Epoch:  1
Loss:  1.2868774
Epoch:  1
Loss:  1.3076533
Epoch:  1
Loss:  1.279764
Epoch:  1
Loss:  1.3625532
Epoch:  1
Loss:  1.2209027
Epoch:  1
Loss:  1.267107
Epoch:  1
Loss:  1.2887871
Epoch:  1
Loss:  1.2736353
Epoch:  1
Loss:  1.2164485
Epoch:  1
Loss:  1.2644155
Epoch:  1
Loss:  1.2501523
Epoch:  1
Loss:  1.3044217
Epoch:  1
Loss:  1.137944
Epoch:  1
Loss:  1.2835139
Epoch:

Epoch:  2
Loss:  0.9875676
Epoch:  2
Loss:  0.8736795
Epoch:  2
Loss:  0.92973775
Epoch:  2
Loss:  0.92970765
Epoch:  2
Loss:  0.94572276
Epoch:  2
Loss:  0.8957101
Epoch:  2
Loss:  0.9618638
Epoch:  2
Loss:  0.8051659
Epoch:  2
Loss:  0.8605119
Epoch:  2
Loss:  0.91499823
Epoch:  2
Loss:  0.8250354
Epoch:  2
Loss:  0.95053864
Epoch:  2
Loss:  0.91655
Epoch:  2
Loss:  1.0152184
Epoch:  2
Loss:  0.91278064
Epoch:  2
Loss:  0.8692544
Epoch:  2
Loss:  0.87575996
Epoch:  2
Loss:  0.87348175
Epoch:  2
Loss:  0.7591796
Epoch:  2
Loss:  0.8692501
Epoch:  2
Loss:  0.77439415
Epoch:  2
Loss:  0.88889074
Epoch:  2
Loss:  0.822292
Epoch:  2
Loss:  0.97038144
Epoch:  2
Loss:  0.86704206
Epoch:  2
Loss:  0.9413435
Epoch:  2
Loss:  0.83672845
Epoch:  2
Loss:  0.9108655
Epoch:  2
Loss:  0.9453999
Epoch:  2
Loss:  0.8273894
Epoch:  2
Loss:  0.87540436
Epoch:  2
Loss:  0.87030154
Epoch:  2
Loss:  0.8652614
Epoch:  2
Loss:  0.9429561
Epoch:  2
Loss:  0.87632036
Epoch:  2
Loss:  0.8663951
Epoch:  2
Loss:

Epoch:  3
Loss:  0.7033736
Epoch:  3
Loss:  0.6801987
Epoch:  3
Loss:  0.6289426
Epoch:  3
Loss:  0.54834056
Epoch:  3
Loss:  0.5456037
Epoch:  3
Loss:  0.6716995
Epoch:  3
Loss:  0.69611156
Epoch:  3
Loss:  0.6397037
Epoch:  3
Loss:  0.6272492
Epoch:  3
Loss:  0.597823
Epoch:  3
Loss:  0.6515621
Epoch:  3
Loss:  0.632841
Epoch:  3
Loss:  0.5548168
Epoch:  3
Loss:  0.65553606
Epoch:  3
Loss:  0.64056194
Epoch:  3
Loss:  0.6640312
Epoch:  3
Loss:  0.67498696
Epoch:  3
Loss:  0.63379645
Epoch:  3
Loss:  0.6012119
Epoch:  3
Loss:  0.5022175
Epoch:  3
Loss:  0.7259823
Epoch:  3
Loss:  0.6273025
Epoch:  3
Loss:  0.61344963
Epoch:  3
Loss:  0.6056131
Epoch:  3
Loss:  0.63023514
Epoch:  3
Loss:  0.6731595
Epoch:  3
Loss:  0.57294935
Epoch:  3
Loss:  0.66914874
Epoch:  3
Loss:  0.6734077
Epoch:  3
Loss:  0.5615334
Epoch:  3
Loss:  0.7132863
Epoch:  3
Loss:  0.67333376
Epoch:  3
Loss:  0.61769575
Epoch:  3
Loss:  0.541195
Epoch:  3
Loss:  0.6743199
Epoch:  3
Loss:  0.51525617
Epoch:  3
Loss:  0

Epoch:  4
Loss:  0.46119776
Epoch:  4
Loss:  0.47985846
Epoch:  4
Loss:  0.41087037
Epoch:  4
Loss:  0.42659706
Epoch:  4
Loss:  0.48169714
Epoch:  4
Loss:  0.41154656
Epoch:  4
Loss:  0.40632167
Epoch:  4
Loss:  0.5153309
Epoch:  4
Loss:  0.4717616
Epoch:  4
Loss:  0.45812422
Epoch:  4
Loss:  0.50447637
Epoch:  4
Loss:  0.45099622
Epoch:  4
Loss:  0.49346146
Epoch:  4
Loss:  0.44322538
Epoch:  4
Loss:  0.4368812
Epoch:  4
Loss:  0.47580805
Epoch:  4
Loss:  0.43470344
Epoch:  4
Loss:  0.52842295
Epoch:  4
Loss:  0.50959325
Epoch:  4
Loss:  0.34307513
Epoch:  4
Loss:  0.41624054
Epoch:  4
Loss:  0.5255777
Epoch:  4
Loss:  0.42608604
Epoch:  4
Loss:  0.5511283
Epoch:  4
Loss:  0.41109976
Epoch:  4
Loss:  0.45426083
Epoch:  4
Loss:  0.44667506
Epoch:  4
Loss:  0.41001478
Epoch:  4
Loss:  0.41445574
Epoch:  4
Loss:  0.39188
Epoch:  4
Loss:  0.37034255
Epoch:  4
Loss:  0.5899805
Epoch:  4
Loss:  0.46515402
Epoch:  4
Loss:  0.42083088
Epoch:  4
Loss:  0.46043307
Epoch:  4
Loss:  0.3575322
Ep

Epoch:  6
Loss:  0.19292736
Epoch:  6
Loss:  0.2993944
Epoch:  6
Loss:  0.18020189
Epoch:  6
Loss:  0.14239082
Epoch:  6
Loss:  0.24745536
Epoch:  6
Loss:  0.2087157
Epoch:  6
Loss:  0.23705046
Epoch:  6
Loss:  0.25962606
Epoch:  6
Loss:  0.2290856
Epoch:  6
Loss:  0.2678493
Epoch:  6
Loss:  0.21218583
Epoch:  6
Loss:  0.23760381
Epoch:  6
Loss:  0.25503725
Epoch:  6
Loss:  0.23461194
Epoch:  6
Loss:  0.23631892
Epoch:  6
Loss:  0.19246018
Epoch:  6
Loss:  0.20373535
Epoch:  6
Loss:  0.18940367
Epoch:  6
Loss:  0.19461216
Epoch:  6
Loss:  0.23274426
Epoch:  6
Loss:  0.25647807
Epoch:  6
Loss:  0.19371705
Epoch:  6
Loss:  0.18862888
Epoch:  6
Loss:  0.24085025
Epoch:  6
Loss:  0.19932052
Epoch:  6
Loss:  0.2250087
Epoch:  6
Loss:  0.309977
Epoch:  6
Loss:  0.1794574
Epoch:  6
Loss:  0.198671
Epoch:  6
Loss:  0.17023894
Epoch:  6
Loss:  0.16558391
Epoch:  6
Loss:  0.2483033
Epoch:  6
Loss:  0.20693335
Epoch:  6
Loss:  0.18010202
Epoch:  6
Loss:  0.19287656
Epoch:  6
Loss:  0.2604346
Epoc

Epoch:  7
Loss:  0.20830862
Epoch:  7
Loss:  0.24121234
Epoch:  7
Loss:  0.18419874
Epoch:  7
Loss:  0.17243919
Epoch:  7
Loss:  0.16538826
Epoch:  7
Loss:  0.13726914
Epoch:  7
Loss:  0.20377699
Epoch:  7
Loss:  0.16516617
Epoch:  7
Loss:  0.1524685
Epoch:  7
Loss:  0.20813975
Epoch:  7
Loss:  0.1382121
Epoch:  7
Loss:  0.16030557
Epoch:  7
Loss:  0.13997518
Epoch:  7
Loss:  0.1739957
Epoch:  7
Loss:  0.15766868
Epoch:  7
Loss:  0.17854595
Epoch:  7
Loss:  0.17639813
Epoch:  7
Loss:  0.15487659
Epoch:  7
Loss:  0.16869637
Epoch:  7
Loss:  0.18455139
Epoch:  7
Loss:  0.16558248
Epoch:  7
Loss:  0.1578925
Epoch:  7
Loss:  0.15222697
Epoch:  7
Loss:  0.17126569
Epoch:  7
Loss:  0.16497517
Epoch:  7
Loss:  0.15921703
Epoch:  7
Loss:  0.19068459
Epoch:  7
Loss:  0.18634054
Epoch:  7
Loss:  0.20437662
Epoch:  7
Loss:  0.14214142
Epoch:  7
Loss:  0.2195693
Epoch:  7
Loss:  0.16475114
Epoch:  7
Loss:  0.12954882
Epoch:  7
Loss:  0.1217041
Epoch:  7
Loss:  0.14979881
Epoch:  7
Loss:  0.2132189

Epoch:  8
Loss:  0.121855274
Epoch:  8
Loss:  0.13394994
Epoch:  8
Loss:  0.108712055
Epoch:  8
Loss:  0.1544557
Epoch:  8
Loss:  0.118401505
Epoch:  8
Loss:  0.13983944
Epoch:  8
Loss:  0.1221752
Epoch:  8
Loss:  0.11407034
Epoch:  8
Loss:  0.12924841
Epoch:  8
Loss:  0.12678441
Epoch:  8
Loss:  0.13570407
Epoch:  8
Loss:  0.15649661
Epoch:  8
Loss:  0.1641331
Epoch:  8
Loss:  0.14225712
Epoch:  8
Loss:  0.15132624
Epoch:  8
Loss:  0.119160965
Epoch:  8
Loss:  0.14520982
Epoch:  8
Loss:  0.17084381
Epoch:  8
Loss:  0.14833054
Epoch:  8
Loss:  0.12948398
Epoch:  8
Loss:  0.1327124
Epoch:  8
Loss:  0.124471806
Epoch:  8
Loss:  0.15894887
Epoch:  8
Loss:  0.16052787
Epoch:  8
Loss:  0.11550214
Epoch:  8
Loss:  0.098348476
Epoch:  8
Loss:  0.15305655
Epoch:  8
Loss:  0.09961872
Epoch:  8
Loss:  0.15961674
Epoch:  8
Loss:  0.10866095
Epoch:  8
Loss:  0.14891748
Epoch:  8
Loss:  0.15356664
Epoch:  8
Loss:  0.17108962
Epoch:  8
Loss:  0.21416955
Epoch:  8
Loss:  0.13044563
Epoch:  8
Loss:  0

Epoch:  9
Loss:  0.119364694
Epoch:  9
Loss:  0.10997097
Epoch:  9
Loss:  0.10661213
Epoch:  9
Loss:  0.106248066
Epoch:  9
Loss:  0.10438739
Epoch:  9
Loss:  0.10026308
Epoch:  9
Loss:  0.0919793
Epoch:  9
Loss:  0.1390322
Epoch:  9
Loss:  0.13265425
Epoch:  9
Loss:  0.09582401
Epoch:  9
Loss:  0.13208245
Epoch:  9
Loss:  0.08475375
Epoch:  9
Loss:  0.098111905
Epoch:  9
Loss:  0.110394694
Epoch:  9
Loss:  0.09150706
Epoch:  9
Loss:  0.09869334
Epoch:  9
Loss:  0.1537759
Epoch:  9
Loss:  0.09288482
Epoch:  9
Loss:  0.12515147
Epoch:  9
Loss:  0.105323754
Epoch:  9
Loss:  0.095630355
Epoch:  9
Loss:  0.104582176
Epoch:  9
Loss:  0.15936138
Epoch:  9
Loss:  0.13481984
Epoch:  9
Loss:  0.10806517
Epoch:  9
Loss:  0.13208431
Epoch:  9
Loss:  0.11128919
Epoch:  9
Loss:  0.08492399
Epoch:  9
Loss:  0.15377262
Epoch:  9
Loss:  0.09390646
Epoch:  9
Loss:  0.10486066
Epoch:  9
Loss:  0.12623331
Epoch:  9
Loss:  0.14873722
Epoch:  9
Loss:  0.1251128
Epoch:  9
Loss:  0.11420442
Epoch:  9
Loss:  

In [87]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [88]:
checkpoint.restore(tf.train.latest_checkpoint(''))

In [89]:
evaluate('hace mucho frio aqui.')

('it s very cold here . <end> ', '<strat> hace mucho frio aqui . <end>')